In [1]:
import numpy as np
import scipy.stats
import os
import re

import statsmodels.formula.api as smf
import statsmodels.api as sm
import statsmodels.stats.multitest


import csv
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Define file paths
data_dir = '../data'
pupil_dir = '../data/SingleTrialPupilData'
eeg_dir = '../data/SingleTrialEEGData'
plot_dir = '../plots'

## Model Fitting

Skip if loading previously fitted data
- Fit to mean pupil dilation for "included" trials

In [3]:
files_pupil = os.listdir(pupil_dir)
files_eeg = os.listdir(eeg_dir)

In [4]:
def function(lr, file, V_0):

    """
    Updates and computes the correlation between V values and mean EEG frequency data.
    
    Parameters:
        - lr (float): 
            Learning rate for updating V values.
        - file (str): 
            Filename of the pupil data CSV.
        - V_0 (list or array): 
            Initial V values.
    
    Returns:
        float: Negative Pearson correlation coefficient between updated V values and mean EEG frequencies.
    """

    # Extract participant identifier from the filename
    res = re.search('(...)_PupilDiameterProcessed', file)
    participant = res.group(1)

    # Find the corresponding EEG file
    match_eeg = [s for s in files_eeg if participant in s]

    # Load the pupil and EEG data
    df_pupil = pd.read_csv(os.path.join(pupil_dir, file))
    df_eeg = pd.read_csv(os.path.join(eeg_dir, match_eeg[0]))

    # Calculate the cumulative number of trials up to each block
    block_1 = np.sum(df_eeg['Block'] == 1)
    block_2 = block_1 + np.sum(df_eeg['Block'] == 2)
    block_3 = block_2 + np.sum(df_eeg['Block'] == 3)
    block_4 = block_3 + np.sum(df_eeg['Block'] == 4)

    # Calculate the mean pupil diameter for each trial
    mean_pupil = np.array(np.mean(df_pupil.loc[:,'0':'3299'], axis = 1))
    df_pupil['means'] = mean_pupil

    # Recode 'side' as numerical values
        # 'side' refers to the direction of the rock
    df_pupil['side'] = (df_pupil['start_new_trial_condition']== 'Left') * 0 + (
        df_pupil['start_new_trial_condition']== 'MiddleLow') * 1 + (
        df_pupil['start_new_trial_condition']== 'Right') * 2 # Recode side as numbers

    # Initialize V values
    V = np.array((V_0))
    all_V = np.zeros((np.shape(df_eeg)[0] + 1, 3))
    all_V[0, :] = V

    # Calculate V values for each trial across blocks
    for block in [1,2,3,4]:

        # Skip block 3 for participant 5, as pain values are missing
        if participant == 'P05' and (block == 3 or block == 4):
            continue
        if participant == 'P14' and block == 4:
            continue
        
        # Get data for block
        this_block = df_eeg[df_eeg['Block'] == block]

        # Update V values for each trial
        for i_t, trial in enumerate(this_block['epoch']):

            # Determine cue side based on 'type'
            if this_block.loc[trial-1, 'type'] == 'left':
                cue_side = 0
            elif this_block.loc[trial-1, 'type'] == 'middle':
                cue_side = 1
            elif this_block.loc[trial-1, 'type'] == 'right':
                cue_side = 2
            else:
                print('Error: Unrecognized cue side.')

            # Determine reward value based on 'pain'
            if this_block.loc[trial-1,'pain'] == 'PCollShock':
                r = 1
            elif this_block.loc[trial-1,'pain'] == 'nPCollNoShock':
                r = 0
            elif this_block.loc[trial-1,'pain'] == 'PCollNoShock':
                r = 0
            elif np.logical_and(np.isnan(this_block.loc[trial-1,'pain']), block == 2 or block == 4):
                r = 0
            else:
                if this_block.loc[trial-1, 'type'] == 'middle':
                    r = 0
                else:
                    r = 1

            # Update V
            V[cue_side] = V[cue_side] * (1 - lr) + lr * r
            all_V[trial, :] = V
    
    # Create a mask to exclude the last trial
    V_incl = ~np.all(df_pupil.loc[:,'0':'3299'] == 0, axis = 1) # Find trials with no missing pupil
    V_incl[np.shape(V_incl)[0] + 1] = False
    pupil_incl = ~np.all(df_pupil.loc[:,'0':'3299'] == 0, axis = 1) # Find trials with no missing data

    
    # Exclude the last trials for participant 5 and 14, as pain values are missing
    if participant == 'P05':
        pupil_incl[180::] = False
        V_incl[180::] = False
    elif participant == 'P14':
        pupil_incl[241::] = False
        V_incl[241::] = False

    # Calculate the correlation between V values and mean pupil diameter
    correlation = scipy.stats.pearsonr(
                all_V[V_incl, df_pupil.loc[pupil_incl, 'side']],
                df_pupil.loc[pupil_incl, 'means'])[0]

    return (-correlation)

In [5]:
# Define boundary constraints for optimization. 
# Lower bound (lb) is set to 0.0 and upper bound (ub) is set to 1.0.
bnds = scipy.optimize.Bounds(lb=0.01, ub=0.99)

# Regression with Congruency

In [7]:
all_tonic_sides = pd.read_csv(os.path.join(data_dir, 'tonicpainsides.csv'))

In [8]:
# Define boundary constraints for optimization. 
# Lower bound (lb) is set to 0.0 and upper bound (ub) is set to 1.0.
bnds = scipy.optimize.Bounds(lb=0.01, ub=0.99)

In [9]:
# Intialize DataFrame to save relevant trial data (pupil, V values, participant ID, and congruency)
all_data = pd.DataFrame()

# Intialize DataFrame to save relevant model fitting data (ID, learning rate, maximum correlation)
df_model = pd.DataFrame()

for i_file, file in enumerate(files_pupil):
    print('========= \n File : {} of {}'.format(i_file + 1, len(files_pupil)))
    try:

        # Initialize V Values for RL
        V_0 = np.array((0.3, 0.3, 0.3)) #np.zeros((3))
        
        # Extract the participant ID from the file name
        ID = int(re.split('_PupilDiameterProcessed_ST.csv', file)[0][1::])

        # Extract the tonic pain sides for the participant
        tonic_sides  = all_tonic_sides.loc[all_tonic_sides['ID'] == ID, 'Extn1' : 'Extn2'].values[0].astype('str')
        tonic_sides[np.where(tonic_sides == '1')] = 'left'
        tonic_sides[np.where(tonic_sides == '2')] = 'right'
        
        # Fit the model
        fit_res = scipy.optimize.minimize(function, (0.5, ), args=(file, V_0), method= 'SLSQP', bounds = bnds)
        print(fit_res.message)
        
        # Extract participant identifier from the filename
        res_file = re.search('(...)_PupilDiameterProcessed', file)
        participant = res_file.group(1)

        # Find the corresponding EEG file
        match_eeg = [s for s in files_eeg if participant in s]

        # Load pupil and EEG data
        df_pupil = pd.read_csv(os.path.join(pupil_dir, file))
        df_eeg = pd.read_csv(os.path.join(eeg_dir, match_eeg[0]))


        # Recode 'side' as numerical values
            # 'side' refers to the direction of the rock
        df_eeg['side'] = (df_eeg['type']== 'left') * 0 + \
            (df_eeg['type']== 'middle') * 1 + \
            (df_eeg['type']== 'right') * 2

        # Calculate the mean pupil diameter for each trial and timebin
        timebin_1 = np.array(np.mean(df_pupil.loc[:,'0':'799'], axis = 1))
        timebin_2 = np.array(np.mean(df_pupil.loc[:,'800':'1799'], axis = 1))
        timebin_3 = np.array(np.mean(df_pupil.loc[:,'1800':'2799'], axis = 1))
        timebin_4 = np.array(np.mean(df_pupil.loc[:,'2800':'3299'], axis = 1))

        df_pupil['t1'] = timebin_1
        df_pupil['t2'] = timebin_2
        df_pupil['t3'] = timebin_3
        df_pupil['t4'] = timebin_4
        
        # Initialize V values
        V = np.array((V_0))
        all_V = np.zeros((np.shape(df_eeg)[0] + 1, 3))
        all_V[0, :] = V
        
        # Save learning rate from model fitting
        lr = fit_res.x
        
        # Calculate V values for each trial across blocks
        for block in [1,2,3,4]:

            # Skip block 3 for participant 5, as pain values are missing
            if ID == 5 and (block == 3 or block == 4):
                print('Skipping block 3 and 4 for participant 5')
                continue
            if ID == 14 and block == 4:
                print('Skipping block 4 for participant 14')
                continue
            
            # Extract data for block
            this_block = df_eeg[df_eeg['Block'] == block]

            # Loop over trials and update V values
            for i_t, trial in enumerate(this_block['epoch']):

                if ID == 14 and trial>241:
                    print('Skipping trial {} in block 3 for participant 14'.format(trial))
                    continue

                # Determine cue side based on 'type'
                if this_block.loc[trial-1, 'type'] == 'left':
                    cue_side = 0
                elif this_block.loc[trial-1, 'type'] == 'middle':
                    cue_side = 1
                elif this_block.loc[trial-1, 'type'] == 'right':
                    cue_side = 2
                else:
                    print('what is going on with the cue sides?')


                # Determine reward value based on 'pain'
                if this_block.loc[trial-1,'pain'] == 'PCollShock':
                    r = 1
                elif this_block.loc[trial-1,'pain'] == 'nPCollNoShock':
                    r = 0
                elif this_block.loc[trial-1,'pain'] == 'PCollNoShock':
                    r = 0
                elif np.logical_and(np.isnan(this_block.loc[trial-1,'pain']), block == 2 or block == 4):
                    r = 0
                else:
                    print('Error: Unrecognized pain value: {}'.format(this_block.loc[trial-1,'pain']))
                    if this_block.loc[trial-1, 'type'] == 'middle':
                        r = 0
                    else:
                        r = 1
                
                # Update V based on reward and learning rate
                V[cue_side] = V[cue_side] * (1 - lr) + lr * r
                all_V[trial, :] = V
        
        # Create a mask for which trials to include
        V_incl = ~np.all(df_pupil.loc[:,'0':'3299'] == 0, axis = 1)
        V_incl[np.shape(V_incl)[0] + 1] = False
        pupil_incl = ~np.all(df_pupil.loc[:,'0':'3299'] == 0, axis = 1)

        # Exclude the last trials for participant 5 and 14, as pain values are missing
        if participant == 'P05':
            pupil_incl[180::] = False
            V_incl[180::] = False
        elif participant == 'P14':
            pupil_incl[241::] = False
            V_incl[241::] = False

        # Create a DataFrame for the V values for the included trials
        df_V = pd.DataFrame(all_V[
            V_incl, 
            df_eeg.loc[pupil_incl, 'side']],
            columns = ['V'])

        # Create a DataFrame for the participant ID (repeated for each trial)
        df_ID = pd.DataFrame(np.repeat(ID, np.shape(df_V)[0]), columns = ['ID'])

        # Filter the pupil data
        df_pupil_in = df_pupil.loc[pupil_incl, 't1': 't4']
        
        # Congruency Information
        cong = np.logical_or(
            np.logical_and(df_eeg['Block'] == 2, df_eeg['type'] == tonic_sides[0]),
            np.logical_and(df_eeg['Block'] == 4, df_eeg['type'] == tonic_sides[1]))
        incong = np.logical_or(
            np.logical_and(df_eeg['Block'] == 2, df_eeg['type'] == tonic_sides[1]),
            np.logical_and(df_eeg['Block'] == 4, df_eeg['type'] == tonic_sides[0]))

        # Create a DataFrame for congruency information (congruent or incongruent on extinction blocks; nan for acquisition blocks)
        df_congr = (np.ones((np.shape(df_eeg)[0])) * np.nan).astype('str')
        df_congr[cong] = "congr" # congruent
        df_congr[incong] = "incongr" # incongruent

        # Filter congruency data for included trials
        df_congr = pd.DataFrame(df_congr[pupil_incl], columns = ['congruency'])
                
        # Add to dataframe
        new_row = pd.concat([df_pupil_in,df_V, df_ID, df_congr], axis = 1)
        all_data = pd.concat([all_data, new_row])

        # Append the model fitting data
        df_model = pd.concat([df_model, pd.DataFrame({'ID': ID, 'lr': lr, 'max_corr': -fit_res.fun})])
    
    except:
        print('__something went wrong')

all_data.reset_index(drop = True)

print('DONE')

 File : 1 of 26
Optimization terminated successfully
 File : 2 of 26
Optimization terminated successfully
 File : 3 of 26
Optimization terminated successfully
 File : 4 of 26
Optimization terminated successfully
Skipping block 3 and 4 for participant 5
Skipping block 3 and 4 for participant 5
 File : 5 of 26
Optimization terminated successfully
 File : 6 of 26
Optimization terminated successfully
 File : 7 of 26
Optimization terminated successfully
 File : 8 of 26
Optimization terminated successfully
 File : 9 of 26
Optimization terminated successfully
Error: Unrecognized pain value: nan
 File : 10 of 26
Optimization terminated successfully
 File : 11 of 26
Optimization terminated successfully
Error: Unrecognized pain value: nan
Error: Unrecognized pain value: nan
 File : 12 of 26
Optimization terminated successfully
Skipping trial 242 in block 3 for participant 14
Skipping trial 243 in block 3 for participant 14
Skipping trial 244 in block 3 for participant 14
Skipping trial 245 in bl

In [10]:
df_model.to_csv(os.path.join(data_dir, 'pupil_model_fitting_results.csv'), index = False)

In [11]:
# Check DataFrame
all_data

,t1,t2,t3,t4,V,ID,congruency
0,0.070559,0.041776,0.247400,0.220276,0.300000,2.0,nan
1,-0.090062,-0.220444,-0.205470,-0.337087,0.300000,2.0,nan
2,0.034449,0.019725,0.280007,0.278826,0.313817,2.0,nan
3,0.032575,-0.051917,-0.075108,-0.124938,0.313817,2.0,nan
4,-0.058258,-0.214787,0.010181,0.163215,0.300000,2.0,nan
...,...,...,...,...,...,...,...
355,-0.080276,-0.251930,-0.316122,-0.298939,0.018540,30.0,incongr
356,-0.051633,-0.171034,-0.031295,-0.021038,0.018700,30.0,congr
357,-0.014987,-0.295661,-0.442957,-0.264412,0.016856,30.0,congr
358,-0.123431,-0.200313,-0.285149,-0.415803,0.000001,30.0,nan


In [12]:
all_data.to_csv(os.path.join(data_dir, 'pupil_v_values.csv'), index = False)

## Load Data

In [13]:
# Read csv file into DataFrame
all_data = pd.read_csv(os.path.join(data_dir, 'pupil_v_values.csv'))

## Regression

In [14]:
# Reshape the 'all_data' DataFrame from wide to long format, using 'ID', 'V', and 'congruency' as identifier variables.
dat = pd.melt(all_data, id_vars = ['ID','V', 'congruency'], var_name = 'timebin', value_name = 'pupil')
dat

,ID,V,congruency,timebin,pupil
0,2.0,0.300000,NaN,t1,0.070559
1,2.0,0.300000,NaN,t1,-0.090062
2,2.0,0.313817,NaN,t1,0.034449
3,2.0,0.313817,NaN,t1,0.032575
4,2.0,0.300000,NaN,t1,-0.058258
...,...,...,...,...,...
36195,30.0,0.018540,incongr,t4,-0.298939
36196,30.0,0.018700,congr,t4,-0.021038
36197,30.0,0.016856,congr,t4,-0.264412
36198,30.0,0.000001,NaN,t4,-0.415803


### Pupil ~ V * timebin

In [15]:
# Performing an ordinary least squares (OLS) regression analysis, modeling 'pupil' as a function of 'V' and 'timebin',including their interaction
res = smf.ols('pupil ~ V*timebin', data = dat).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  pupil   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     119.0
Date:                Mon, 09 Sep 2024   Prob (F-statistic):          1.37e-173
Time:                        20:06:01   Log-Likelihood:                -12541.
No. Observations:               35696   AIC:                         2.510e+04
Df Residuals:                   35688   BIC:                         2.517e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.0086      0.004     -1.908      0.056      -0.017       0.000
timebin[T.t2]      -0.1097      0.006    -17.261      0.000      -0.122      -0.097
timebin[T.t3]      -0.0385      0.006     -6.053      0.000      -0.051      -0.026
timebin[T.t4]       0.0005      0.006      0.079      0.937      -0.012       0.013
V                   0.0112      0.012      0.948      0.343      -0.012       0.034
V:timebin[T.t2]     0.0105      0.017      0.629      0.529      -0.022       0.043
V:timebin[T.t3]     0.0531      0.017      3.192      0.001       0.021       0.086
V:timebin[T.t4]     0.0939      0.017      5.637      0.000       0.061       0.126
==============================================================================
Omnibus:                     8775.540   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           311339.065
Skew:                          -0.487   Prob(JB):                         0.00
Kurtosis:                      17.435   Cond. No.                         16.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
# Extract the p-values from the OLS regression result.
p_values = res.pvalues 

# Perform multiple hypothesis testing correction on the p-values using the Benjamini-Hochberg method (FDR correction), with a significance level (alpha) of 0.05.
_, pvals_corrected, _, _ = statsmodels.stats.multitest.multipletests(p_values, alpha=0.05, method='fdr_bh')

# Convert the original and corrected p-values to a DataFrame for better visualization.
dat_p = p_values.to_frame()
dat_p.insert(1, "corrected", pvals_corrected)

# Save the DataFrame to an Excel file.
dat_p.to_excel("output_pupil_V-timebin.xlsx")
dat_p

,0,corrected
Intercept,5.644075e-02,9.030520e-02
timebin[T.t2],1.736329e-66,1.389063e-65
timebin[T.t3],1.434959e-09,5.739836e-09
timebin[T.t4],9.371619e-01,9.371619e-01
V,3.430942e-01,4.574590e-01
V:timebin[T.t2],5.291822e-01,6.047797e-01
V:timebin[T.t3],1.414005e-03,2.828011e-03
V:timebin[T.t4],1.742258e-08,4.646021e-08


### Pupil ~ V*congruency

In [17]:
dat_cong = dat[~(dat['congruency'].values == 'nan')]
res = smf.ols('pupil ~ V*congruency', data = dat_cong).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  pupil   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     11.54
Date:                Mon, 09 Sep 2024   Prob (F-statistic):           1.49e-07
Time:                        20:06:05   Log-Likelihood:                -4895.0
No. Observations:               14184   AIC:                             9798.
Df Residuals:                   14180   BIC:                             9828.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  -0.0539      0.005    -11.146      0.000      -0.063      -0.044
congruency[T.incongr]      -0.0019      0.007     -0.275      0.783      -0.015       0.012
V                           0.1006      0.021      4.698      0.000       0.059       0.143
V:congruency[T.incongr]    -0.0287      0.030     -0.957      0.339      -0.088       0.030
==============================================================================
Omnibus:                     3120.823   Durbin-Watson:                   1.910
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            69326.577
Skew:                          -0.507   Prob(JB):                         0.00
Kurtosis:                      13.783   Cond. No.                         14.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
# Extract the p-values from the OLS regression result.
p_values = res.pvalues

# Perform multiple hypothesis testing correction on the p-values using the Benjamini-Hochberg method (FDR correction), with a significance level (alpha) of 0.05.
_, pvals_corrected, _, _ = statsmodels.stats.multitest.multipletests(p_values, alpha=0.05, method='fdr_bh')

# Convert the original and corrected p-values to a DataFrame for better visualization.
dat_p = p_values.to_frame()
dat_p.insert(1, "corrected", pvals_corrected)

# Save the DataFrame with original and corrected p-values to an Excel file.
dat_p.to_excel("output_pupil_V-congruency.xlsx")
dat_p

,0,corrected
Intercept,9.808218e-29,3.923287e-28
congruency[T.incongr],7.831892e-01,7.831892e-01
V,2.657254e-06,5.314508e-06
V:congruency[T.incongr],3.387692e-01,4.516922e-01


## Pupil ~ V * timebin * congruency

In [19]:
dat_cong = dat[~(dat['congruency'].values == 'nan')]
res = smf.ols('pupil ~ V*timebin*congruency', data = dat_cong).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  pupil   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     21.59
Date:                Mon, 09 Sep 2024   Prob (F-statistic):           3.30e-59
Time:                        20:06:21   Log-Likelihood:                -4752.0
No. Observations:               14184   AIC:                             9536.
Df Residuals:                   14168   BIC:                             9657.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                -0.0136      0.010     -1.417      0.157      -0.032       0.005
timebin[T.t2]                            -0.1233      0.014     -9.096      0.000      -0.150      -0.097
timebin[T.t3]                            -0.0395      0.014     -2.915      0.004      -0.066      -0.013
timebin[T.t4]                             0.0014      0.014      0.102      0.919      -0.025       0.028
congruency[T.incongr]                    -0.0098      0.014     -0.726      0.468      -0.036       0.017
timebin[T.t2]:congruency[T.incongr]       0.0107      0.019      0.559      0.576      -0.027       0.048
timebin[T.t3]:congruency[T.incongr]       0.0126      0.019      0.659      0.510      -0.025       0.050
timebin[T.t4]:congruency[T.incongr]       0.0085      0.019      0.444      0.657      -0.029       0.046
V                                         0.0318      0.042      0.750      0.453      -0.051       0.115
V:timebin[T.t2]                           0.0849      0.060      1.415      0.157      -0.033       0.203
V:timebin[T.t3]                           0.0710      0.060      1.184      0.237      -0.047       0.189
V:timebin[T.t4]                           0.1193      0.060      1.987      0.047       0.002       0.237
V:congruency[T.incongr]                   0.0206      0.059      0.347      0.728      -0.096       0.137
V:timebin[T.t2]:congruency[T.incongr]    -0.0224      0.084     -0.266      0.790      -0.187       0.142
V:timebin[T.t3]:congruency[T.incongr]    -0.0846      0.084     -1.006      0.314      -0.249       0.080
V:timebin[T.t4]:congruency[T.incongr]    -0.0904      0.084     -1.075      0.282      -0.255       0.074
==============================================================================
Omnibus:                     3082.707   Durbin-Watson:                   1.948
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            70401.271
Skew:                          -0.480   Prob(JB):                         0.00
Kurtosis:                      13.872   Cond. No.                         66.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
# Extract the p-values from the OLS regression result.
p_values = res.pvalues

# Perform multiple hypothesis testing correction on the p-values using the Benjamini-Hochberg method (FDR correction), with a significance level (alpha) of 0.05.
_, pvals_corrected, _, _ = statsmodels.stats.multitest.multipletests(p_values, alpha=0.05, method='fdr_bh')

# Convert the original and corrected p-values to a DataFrame for better visualization.
dat_p = p_values.to_frame()
dat_p.insert(1, "corrected", pvals_corrected)

# Save the DataFrame with original and corrected p-values to an Excel file.
dat_p.to_excel("output_pupil_V-timebin-congruency.xlsx")
dat_p

,0,corrected
Intercept,1.566087e-01,5.023359e-01
timebin[T.t2],1.056613e-19,1.690581e-18
timebin[T.t3],3.562796e-03,2.850237e-02
timebin[T.t4],9.189441e-01,9.189441e-01
congruency[T.incongr],4.676323e-01,7.415256e-01
timebin[T.t2]:congruency[T.incongr],5.763354e-01,7.684472e-01
timebin[T.t3]:congruency[T.incongr],5.097988e-01,7.415256e-01
timebin[T.t4]:congruency[T.incongr],6.573429e-01,8.090375e-01
V,4.532198e-01,7.415256e-01
V:timebin[T.t2],1.569800e-01,5.023359e-01
